In [1]:
import pandas as pd
import numpy as np

import csv
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import seaborn as sb

from scipy import stats
import requests
from datetime import datetime

### Acquisition

I acquired the data for Average Market Rent from Bigger Pockets, a media company and social network focused on commercial and residential real estate investing. This particular dataset was created by Kaylin Cooper, an investor who has been tracking market trends in the top 100 cities in the U.S. since 2017. 

Size at Acquisition: 300 rows x 77 columns
Source: https://www.biggerpockets.com/files/user/leahd42/file/webinar-bonus-2022-rental-data

The original excel sheet had all of the cities merged into 3 cells, then the next column indicated the difference between 1, 2, and 3 bedroom properties. That made for easier readability as a spreadsheet, but it translated a lot of NaN values into the dataframe. The first step I had to take was unmerging all of those cells, then imputing the names of each city into their corresponding bedroom count.

In [2]:
amr = pd.read_csv('avg_market_rent_HUD.csv')

In [3]:
# Original
amr.head(10)

,Largest 100 Cities,Beds,1/1/2017,2/1/2017,3/1/2017,4/1/2017,5/1/2017,6/1/2017,7/1/2017,8/1/2017,...,6/1/2022,7/1/2022,8/1/2022,9/1/2022,10/1/2022,11/1/2022,12/1/2022,1/1/2023,2/1/2023,3/1/2023
0,"ALBUQUERQUE, NM",1,$660,$659,$670,$675,$675,$690,$677,$675,...,"$1,176","$1,185","$1,215","$1,204","$1,164","$1,145","$1,175","$1,184","$1,165","$1,151"
1,NaN,2,$819,$800,$817,$825,$834,$875,$865,$850,...,"$1,478","$1,475","$1,489","$1,501","$1,500","$1,480","$1,450","$1,469","$1,450","$1,428"
2,NaN,3,"$1,175","$1,195","$1,150","$1,200","$1,200","$1,226","$1,225","$1,225",...,"$1,800","$1,872","$1,850","$1,850","$1,900","$1,875","$1,800","$1,857","$1,850","$1,942"
3,"ANAHEIM, CA",1,"$1,480","$1,500","$1,510","$1,516","$1,505","$1,525","$1,524","$1,535",...,"$2,300","$2,312","$2,305","$2,340","$2,270","$2,225","$2,260","$2,529","$2,279","$2,229"
4,NaN,2,"$1,840","$1,813","$1,830","$1,835","$1,850","$1,865","$1,870","$1,855",...,"$2,775","$2,704","$2,731","$2,769","$2,820","$2,743","$2,755","$2,882","$2,845","$2,700"
5,NaN,3,"$2,363","$2,295","$2,300","$2,500","$2,400","$2,450","$2,500","$2,598",...,"$3,203","$3,203","$3,000","$3,350","$3,600","$3,600","$3,600","$3,600","$3,500","$3,400"
6,"ANCHORAGE, AK",1,$910,$900,$900,$895,$890,$895,$875,$875,...,"$1,165","$1,190","$1,196","$1,200","$1,240","$1,198","$1,200","$1,188","$1,195","$1,200"
7,NaN,2,"$1,150","$1,110","$1,100","$1,133","$1,090","$1,085","$1,100","$1,100",...,"$1,450","$1,453","$1,495","$1,500","$1,450","$1,425","$1,450","$1,450","$1,450","$1,405"
8,NaN,3,"$1,650","$1,575","$1,550","$1,600","$1,545","$1,550","$1,630","$1,675",...,"$1,998","$1,950","$1,950","$2,000","$2,000","$2,000","$2,000","$1,900","$1,900","$1,900"
9,"ARLINGTON, TX",1,$750,$746,$760,$765,$770,$780,$784,$775,...,"$1,210","$1,235","$1,228","$1,233","$1,204","$1,195","$1,235","$1,234","$1,200","$1,223"


In [4]:
df = pd.read_csv('HUDpro_amr_unmerged.csv')

In [5]:
# Unmerged
df

,City,Beds,1/1/2017,2/1/2017,3/1/2017,4/1/2017,5/1/2017,6/1/2017,7/1/2017,8/1/2017,...,6/1/2022,7/1/2022,8/1/2022,9/1/2022,10/1/2022,11/1/2022,12/1/2022,1/1/2023,2/1/2023,3/1/2023
0,"ALBUQUERQUE, NM",1,$660,$659,$670,$675,$675,$690,$677,$675,...,"$1,176","$1,185","$1,215","$1,204","$1,164","$1,145","$1,175","$1,184","$1,165","$1,151"
1,"ALBUQUERQUE, NM",2,$819,$800,$817,$825,$834,$875,$865,$850,...,"$1,478","$1,475","$1,489","$1,501","$1,500","$1,480","$1,450","$1,469","$1,450","$1,428"
2,"ALBUQUERQUE, NM",3,"$1,175","$1,195","$1,150","$1,200","$1,200","$1,226","$1,225","$1,225",...,"$1,800","$1,872","$1,850","$1,850","$1,900","$1,875","$1,800","$1,857","$1,850","$1,942"
3,"ANAHEIM, CA",1,"$1,480","$1,500","$1,510","$1,516","$1,505","$1,525","$1,524","$1,535",...,"$2,300","$2,312","$2,305","$2,340","$2,270","$2,225","$2,260","$2,529","$2,279","$2,229"
4,"ANAHEIM, CA",2,"$1,840","$1,813","$1,830","$1,835","$1,850","$1,865","$1,870","$1,855",...,"$2,775","$2,704","$2,731","$2,769","$2,820","$2,743","$2,755","$2,882","$2,845","$2,700"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,"WICHITA, KS",2,$675,$660,$675,$675,$675,$681,$675,$680,...,$815,$849,$853,$853,$849,$850,$850,$825,$849,$850
296,"WICHITA, KS",3,$875,$839,$895,$900,$800,$809,$825,$850,...,"$1,038","$1,050","$1,150","$1,125","$1,100","$1,095","$1,150","$1,158","$1,250","$1,250"
297,"WINSTON SALEM, NC",1,$720,$725,$728,$749,$774,$760,$750,$730,...,"$1,180","$1,223","$1,205","$1,160","$1,104","$1,250","$1,230","$1,243","$1,157","$1,170"
298,"WINSTON SALEM, NC",2,$775,$795,$790,$795,$797,$812,$795,$795,...,"$1,360","$1,405","$1,379","$1,338","$1,255","$1,250","$1,255","$1,265","$1,295","$1,275"


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 77 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   City       300 non-null    object
 1   Beds       300 non-null    int64 
 2   1/1/2017   300 non-null    object
 3   2/1/2017   300 non-null    object
 4   3/1/2017   300 non-null    object
 5   4/1/2017   300 non-null    object
 6   5/1/2017   300 non-null    object
 7   6/1/2017   300 non-null    object
 8   7/1/2017   300 non-null    object
 9   8/1/2017   300 non-null    object
 10  9/1/2017   300 non-null    object
 11  10/1/2017  300 non-null    object
 12  11/1/2017  300 non-null    object
 13  12/1/2017  300 non-null    object
 14  1/1/2018   300 non-null    object
 15  2/1/2018   300 non-null    object
 16  3/1/2018   300 non-null    object
 17  4/1/2018   300 non-null    object
 18  5/1/2018   300 non-null    object
 19  6/1/2018   300 non-null    object
 20  7/1/2018   300 non-null    objec

### Preparation

For the sake of reaching MVP, we will only be looking at two bedroom units in San Antonio. We'll have to concat this average market rent (AMR) data with HUD's Fair Market Rent (FMR) data. Here are the steps that we'll take to make sure the dataframes match:

- Set the dates as the index for the dataframe
- Isolate the market rent for the San Antonio market only
- Keep only the values for 2 bedroom units


In [7]:
# Create a dataframe that only contains information for San Antonio
sanant = df.loc[(df['City']== 'SAN ANTONIO, TX')].T
                        

In [8]:
sanant

,246,247,248
City,"SAN ANTONIO, TX","SAN ANTONIO, TX","SAN ANTONIO, TX"
Beds,1,2,3
1/1/2017,$806,"$1,019","$1,310"
2/1/2017,$800,"$1,014","$1,345"
3/1/2017,$824,"$1,040","$1,350"
...,...,...,...
11/1/2022,"$1,200","$1,495","$1,750"
12/1/2022,"$1,204","$1,502","$1,750"
1/1/2023,"$1,194","$1,502","$1,770"
2/1/2023,"$1,196","$1,500","$1,760"


In [9]:
sanant.columns

Int64Index([246, 247, 248], dtype='int64')

In [10]:
sanant.drop(index='City')

,246,247,248
Beds,1,2,3
1/1/2017,$806,"$1,019","$1,310"
2/1/2017,$800,"$1,014","$1,345"
3/1/2017,$824,"$1,040","$1,350"
4/1/2017,$820,"$1,044","$1,375"
...,...,...,...
11/1/2022,"$1,200","$1,495","$1,750"
12/1/2022,"$1,204","$1,502","$1,750"
1/1/2023,"$1,194","$1,502","$1,770"
2/1/2023,"$1,196","$1,500","$1,760"


In [11]:
# Create a dataframe that only return information for 2 bedroom units in San Antonio
# Drop 'City' and 'Beds' from dataframe
# Label the axis 'Date'
sanant_amr = sanant.rename(columns={246:'1 Bed', 247:'AMR', 248:'3 Bed'}).drop(index=['City', 'Beds'], columns= ['1 Bed', '3 Bed']).rename_axis('Date')

In [12]:
sanant_amr

,AMR
Date,
1/1/2017,"$1,019"
2/1/2017,"$1,014"
3/1/2017,"$1,040"
4/1/2017,"$1,044"
5/1/2017,"$1,012"
...,...
11/1/2022,"$1,495"
12/1/2022,"$1,502"
1/1/2023,"$1,502"


In [13]:
# Remove symbols from AMR
sanant_amr['AMR']= sanant_amr['AMR'].str.replace('$','').str.replace(',','')
sanant_amr

/var/folders/3f/gcykrn_x5kj_vxldk0lkx7p00000gn/T/ipykernel_1062/2953927494.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sanant_amr['AMR']= sanant_amr['AMR'].str.replace('$','').str.replace(',','')


,AMR
Date,
1/1/2017,1019
2/1/2017,1014
3/1/2017,1040
4/1/2017,1044
5/1/2017,1012
...,...
11/1/2022,1495
12/1/2022,1502
1/1/2023,1502


In [14]:
# Convert AMR to integer
sanant_amr['AMR'] = sanant_amr['AMR'].astype('int')

In [15]:
np.dtype(sanant_amr['AMR'])

dtype('int64')

In [16]:
sanant_amr.index

Index(['1/1/2017', '2/1/2017', '3/1/2017', '4/1/2017', '5/1/2017', '6/1/2017',
       '7/1/2017', '8/1/2017', '9/1/2017', '10/1/2017', '11/1/2017',
       '12/1/2017', '1/1/2018', '2/1/2018', '3/1/2018', '4/1/2018', '5/1/2018',
       '6/1/2018', '7/1/2018', '8/1/2018', '9/1/2018', '10/1/2018',
       '11/1/2018', '12/1/2018', '1/1/2019', '2/1/2019', '3/1/2019',
       '4/1/2019', '5/1/2019', '6/1/2019', '7/1/2019', '8/1/2019', '9/1/2019',
       '10/1/2019', '11/1/2019', '12/1/2019', '1/1/2020', '2/1/2020',
       '3/1/2020', '4/1/2020', '5/1/2020', '6/1/2020', '7/1/2020', '8/1/2020',
       '9/1/2020', '10/1/2020', '11/1/2020', '12/1/2020', '1/1/2021',
       '2/1/2021', '3/1/2021', '4/1/2021', '5/1/2021', '6/1/2021', '7/1/2021',
       '8/1/2021', '9/1/2021', '10/1/2021', '11/1/2021', '12/1/2021',
       '1/1/2022', '2/1/2022', '3/1/2022', '4/1/2022', '5/1/2022', '6/1/2022',
       '7/1/2022', '8/1/2022', '9/1/2022', '10/1/2022', '11/1/2022',
       '12/1/2022', '1/1/2023', '2/1/202

In [17]:
# My dataframe isn't recognizing the Dates as a datetime index, so I need to fix that

# Create a list of the date strings.
date_strings = ['1/1/2017', '2/1/2017', '3/1/2017', '4/1/2017', '5/1/2017', '6/1/2017', '7/1/2017', '8/1/2017', '9/1/2017', '10/1/2017', '11/1/2017', '12/1/2017', '1/1/2018', '2/1/2018', '3/1/2018', '4/1/2018', '5/1/2018', '6/1/2018', '7/1/2018', '8/1/2018', '9/1/2018', '10/1/2018', '11/1/2018', '12/1/2018', '1/1/2019', '2/1/2019', '3/1/2019', '4/1/2019', '5/1/2019', '6/1/2019', '7/1/2019', '8/1/2019', '9/1/2019', '10/1/2019', '11/1/2019', '12/1/2019', '1/1/2020', '2/1/2020', '3/1/2020', '4/1/2020', '5/1/2020', '6/1/2020', '7/1/2020', '8/1/2020', '9/1/2020', '10/1/2020', '11/1/2020', '12/1/2020', '1/1/2021', '2/1/2021', '3/1/2021', '4/1/2021', '5/1/2021', '6/1/2021', '7/1/2021', '8/1/2021', '9/1/2021', '10/1/2021', '11/1/2021', '12/1/2021', '1/1/2022', '2/1/2022', '3/1/2022', '4/1/2022', '5/1/2022', '6/1/2022', '7/1/2022', '8/1/2022', '9/1/2022', '10/1/2022', '11/1/2022', '12/1/2022', '1/1/2023', '2/1/2023', '3/1/2023']

# Create a pandas DatetimeIndex using the list of date strings.
date_index = pd.DatetimeIndex(date_strings)

# Print the resulting datetime index.
print(date_index)


DatetimeIndex(['2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01',
               '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01',
               '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01',
               '2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01',
               '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01',
               '2018-09-01', '2018-10-01', '2018-11-01', '2018-12-01',
               '2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
               '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01',
               '2019-09-01', '2019-10-01', '2019-11-01', '2019-12-01',
               '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01',
               '2020-05-01', '2020-06-01', '2020-07-01', '2020-08-01',
               '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01',
               '2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01',
               '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01',
      

In [18]:
# Set my index as date_index 
sanant_amr = sanant_amr.set_index(date_index)

In [20]:
sanant_amr

,AMR
2017-01-01,1019
2017-02-01,1014
2017-03-01,1040
2017-04-01,1044
2017-05-01,1012
...,...
2022-11-01,1495
2022-12-01,1502
2023-01-01,1502
2023-02-01,1500


In [21]:
def get_sanant_amr_data():
    """ """
    #Acquire the data from csv
    df = pd.read_csv('HUDpro_amr_unmerged.csv')
    
    # Create a dataframe that only contains information for San Antonio
    df = df.loc[(df['City']== 'SAN ANTONIO, TX')].T
    # Create a dataframe that only return information for 2 bedroom units in San Antonio
    # Drop 'City' and 'Beds' from dataframe
    # Label the axis 'Date'
    df = df.rename(columns={246:'1 Bed', 247:'AMR', 248:'3 Bed'}).drop(index=
                        ['City', 'Beds'], columns= ['1 Bed', '3 Bed']).rename_axis('Date')
    
    # Create a list of the date strings.
    date_strings = ['1/1/2017', '2/1/2017', '3/1/2017', '4/1/2017', '5/1/2017', '6/1/2017', '7/1/2017', '8/1/2017', '9/1/2017', '10/1/2017', '11/1/2017', '12/1/2017', '1/1/2018', '2/1/2018', '3/1/2018', '4/1/2018', '5/1/2018', '6/1/2018', '7/1/2018', '8/1/2018', '9/1/2018', '10/1/2018', '11/1/2018', '12/1/2018', '1/1/2019', '2/1/2019', '3/1/2019', '4/1/2019', '5/1/2019', '6/1/2019', '7/1/2019', '8/1/2019', '9/1/2019', '10/1/2019', '11/1/2019', '12/1/2019', '1/1/2020', '2/1/2020', '3/1/2020', '4/1/2020', '5/1/2020', '6/1/2020', '7/1/2020', '8/1/2020', '9/1/2020', '10/1/2020', '11/1/2020', '12/1/2020', '1/1/2021', '2/1/2021', '3/1/2021', '4/1/2021', '5/1/2021', '6/1/2021', '7/1/2021', '8/1/2021', '9/1/2021', '10/1/2021', '11/1/2021', '12/1/2021', '1/1/2022', '2/1/2022', '3/1/2022', '4/1/2022', '5/1/2022', '6/1/2022', '7/1/2022', '8/1/2022', '9/1/2022', '10/1/2022', '11/1/2022', '12/1/2022', '1/1/2023', '2/1/2023', '3/1/2023']
    # Create a pandas DatetimeIndex using the list of date strings.
    date_index = pd.DatetimeIndex(date_strings)
    # Set my index as date_index 
    df = df.set_index(date_index)
    
    # Remove noise non-integer values from AMR column
    df['AMR']= df['AMR'].str.replace('$','').str.replace(',','')
    
    return df

In [22]:
get_sanant_amr_data()

/var/folders/3f/gcykrn_x5kj_vxldk0lkx7p00000gn/T/ipykernel_1062/2407356964.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['AMR']= df['AMR'].str.replace('$','').str.replace(',','')


,AMR
2017-01-01,1019
2017-02-01,1014
2017-03-01,1040
2017-04-01,1044
2017-05-01,1012
...,...
2022-11-01,1495
2022-12-01,1502
2023-01-01,1502
2023-02-01,1500


Now we have a dataframe that should be compatible with the HUD FMR dataframe. Our next step will be to merge them, then we should be good to begin exploration. 